# Construyendo tu Red Neuronal Profunda: Paso a Paso

En este cuaderno, implementarás todas las funciones necesarias para construir una red neuronal profunda.


**Después de este cuaderno, serás capaz de:**
- Usar unidades no lineales como ReLU para mejorar tu modelo.
- Construir una red neuronal más profunda (con más de una capa oculta).
- Implementar una clase de red neuronal fácil de usar.

**Notación**:
- El superíndice $[l]$ denota una cantidad asociada con la $l^{ésima}$ capa.  
    - Ejemplo: $a^{[L]}$ es la activación de la capa $L$. $W^{[L]}$ y $b^{[L]}$ son los parámetros de la capa $L$.
- El superíndice $(i)$ denota una cantidad asociada con el $i^{ésimo}$ ejemplo.  
    - Ejemplo: $x^{(i)}$ es el $i^{ésimo}$ ejemplo de entrenamiento.
- El subíndice $i$ denota la $i^{ésima}$ entrada de un vector.  
    - Ejemplo: $a^{[l]}_i$ denota la $i^{ésima}$ entrada de las activaciones de la capa $l$.


## 1 - Paquetes  

Primero, importemos todos los paquetes necesarios.  

- [numpy](www.numpy.org) es el paquete principal para la computación científica con Python.  
- [matplotlib](http://matplotlib.org) es una biblioteca para graficos en Python.  
- `dnn_utils` proporciona algunas funciones necesarias para este cuaderno.  
- `testCases` proporciona algunos casos de prueba para evaluar la corrección de tus funciones.  
- `np.random.seed(1)` se usa para mantener consistentes todas las llamadas a funciones aleatorias. Esto nos ayudará a calificar tu trabajo. Por favor, no cambies la semilla.

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v2 import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

## 2 - Esquema del cuaderno 

Para construir tu red neuronal, implementarás varias "funciones auxiliares". Estas funciones auxiliares se utilizarán en la próxima asignación para construir una red neuronal de dos capas y una red neuronal de L capas. Cada pequeña función auxiliar que implementarás tendrá instrucciones detalladas que te guiarán a través de los pasos necesarios. Aquí tienes un esquema de esta asignación, en la que harás lo siguiente:  

- Inicializar los parámetros para una red neuronal de dos capas y para una red neuronal de L capas.  
- Implementar el módulo de propagación hacia adelante.  
     - Completar la parte **LINEAL** de la propagación hacia adelante de una capa (resultando en $ Z^{[l]} $).  
     - Se te proporcionará la función de **ACTIVACIÓN** (ReLU/Sigmoide).  
     - Combinar los dos pasos anteriores en una nueva función de propagación hacia adelante **[LINEAR->ACTIVATION]**.  
     - Apilar la función de propagación hacia adelante **[LINEAR->RELU]** L-1  veces (para las capas 1 a  L-1 ) y agregar **[LINEAR->SIGMOID]** al final (para la capa final L ). Esto te dará una nueva función **L_model_forward**.  
- Calcular la función de pérdida (loss).  
- Implementar el módulo de propagación hacia atrás (indicado en rojo en la figura a continuación).  
    - Completar la parte **LINEAL** de la propagación hacia atrás de una capa.  
    - Se te proporcionará el gradiente de la función **ACTIVACIÓN** (relu_backward/sigmoid_backward).  
    - Combinar los dos pasos anteriores en una nueva función de propagación hacia atrás **[LINEAR->ACTIVATION]**.  
    - Apilar **[LINEAR->RELU]** hacia atrás L-1 veces y agregar **[LINEAR->SIGMOID]** hacia atrás en una nueva función **L_model_backward**.  
- Finalmente, actualizar los parámetros.  


**Nota:** Para cada función de propagación hacia adelante, existe una función correspondiente de propagación hacia atrás. Por ello, en cada paso del módulo de propagación hacia adelante almacenarás algunos valores en una memoria intermedia (**cache**). Estos valores almacenados serán útiles para calcular los gradientes. Luego, en el módulo de retropropagación, utilizarás la memoria intermedia para calcular los gradientes. Esta asignación te mostrará exactamente cómo llevar a cabo cada uno de estos pasos.

### 3.2 - Inicializar  

La inicialización para una red neuronal profunda de $L$ capas es más compleja porque hay muchas más matrices de pesos y vectores de sesgo. Al completar `initialize_parameters_deep`, debes asegurarte de que las dimensiones coincidan entre cada capa. Recuerda que $ n^{[l]} $ es el número de unidades en la capa $l$. Por ejemplo, si el tamaño de nuestra entrada $X$ es $(12288, 209)$ (con $m = 209$ ejemplos), entonces:

| **Capa** | **Forma de $ W $** | **Forma de $ b $** | **Fórmula de Activación** |
|----------|---------------------|---------------------|-----------------------------|
| 1        | $ (n^{[1]}, 12288) $ | $ (n^{[1]}, 1) $ | $ Z^{[1]} = W^{[1]} X + b^{[1]} $ |
| 2        | $ (n^{[2]}, n^{[1]}) $ | $ (n^{[2]},1) $ | $ Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]} $ |
| $ L-1 $ | $ (n^{[L-1]}, n^{[L-2]}) $ | $ (n^{[L-1]}, 1) $ | $ Z^{[L-1]} = W^{[L-1]} A^{[L-2]} + b^{[L-1]} $ |
| $ L $ | $ (n^{[L]}, n^{[L-1]}) $ | $ (n^{[L]}, 1) $ | $ Z^{[L]} = W^{[L]} A^{[L-1]} + b^{[L]} $ |

Recuerda que cuando computamos $ WX + b $ en Python, se realiza automáticamente la difusión de valores (**broadcasting**). Por ejemplo, si:

$$
W = \begin{bmatrix}
    j  & k  & l\\
    m  & n  & o \\  
    p  & q  & r  
\end{bmatrix}, \quad 
X = \begin{bmatrix}
    a  & b  & c\\
    d  & e  & f \\  
    g  & h  & i  
\end{bmatrix}, \quad  
b =\begin{bmatrix}
    s  \\  
    t  \\  
    u  
\end{bmatrix}
$$

Entonces, $ WX + b $ será:

$$
WX + b = \begin{bmatrix}
    (ja + kd + lg) + s  & (jb + ke + lh) + s  & (jc + kf + li)+ s\\
    (ma + nd + og) + t & (mb + ne + oh) + t & (mc + nf + oi) + t\\
    (pa + qd + rg) + u & (pb + qe + rh) + u & (pc + qf + ri)+ u
\end{bmatrix}
$$

### **Ejercicio**: Implementar la inicialización para una Red Neuronal de $ L $ capas.  

### **Instrucciones**:  
- La estructura del modelo es *[LINEAR -> RELU] $ \times (L-1) $ -> LINEAR -> SIGMOIDE*. Es decir, tiene $ L-1 $ capas con una función de activación **ReLU**, seguidas de una capa de salida con activación **sigmoide**.  
- Usa inicialización aleatoria para las matrices de pesos. Utiliza `np.random.rand(shape) * 0.01`.  
- Usa inicialización en ceros para los sesgos. Utiliza `np.zeros(shape)`.  
- Almacenaremos $ n^{[l]} $, el número de unidades en las diferentes capas, en una variable llamada `layer_dims`.  
  - Por ejemplo, si `layer_dims` fuera `[2,4,1]`. Esto significa:  
    - **2 entradas**  
    - **1 capa oculta con 4 unidades ocultas**  
    - **1 capa de salida con 1 unidad de salida**  
  - Como resultado, las dimensiones de los pesos y sesgos fueron:  
    - `W1`: $ (4,2) $  
    - `b1`: $ (4,1) $  
    - `W2`: $ (1,4) $  
    - `b2`: $ (1,1) $  
  - Ahora, generalizaremos esto para $ L $ capas.  

- A continuación, se muestra la implementación para $ L=1 $ (una red neuronal de una sola capa). Usa esto como referencia para implementar el caso general (red neuronal de $ L $ capas).  

```python
    if L == 1:
        parameters["W" + str(L)] = np.random.randn(layer_dims[1], layer_dims[0]) * 0.01
        parameters["b" + str(L)] = np.zeros((layer_dims[1], 1))
```


In [2]:
# FUNCTION: initialize_parameters_deep

def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        
        ### END CODE HERE ###
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [ ]:
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Salida esperada**:
       
<table style="width:80%">
  <tr>
    <td> **W1** </td>
    <td>[[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]</td> 
  </tr>
  
  <tr>
    <td>**b1** </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]</td> 
  </tr>
  
  <tr>
    <td>**W2** </td>
    <td>[[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]</td> 
  </tr>
  
  <tr>
    <td>**b2** </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]]</td> 
  </tr>
  
</table>

## 4 - Módulo de Propagación Hacia Adelante  

### 4.1 - Propagación Lineal  

Ahora que has inicializado tus parámetros, procederás con el módulo de propagación hacia adelante. Comenzarás implementando algunas funciones básicas que utilizarás más adelante para construir el modelo completo. Implementarás tres funciones en el siguiente orden:  

- **LINEAR**  
- **LINEAR -> ACTIVATION**, donde **ACTIVATION** será **ReLU** o **Sigmoide**.  
- **[LINEAR -> RELU] $ \times (L-1) $ -> LINEAR -> SIGMOIDE** (modelo completo).  

El módulo de propagación lineal (vectorizado sobre todos los ejemplos) calcula la siguiente ecuación:  


$$ Z^{[l]} = W^{[l]}A^{[l-1]} + b^{[l]} $$

donde $ A^{[0]} = X $.  

### **Ejercicio**: Construir la parte lineal de la propagación hacia adelante.  

### **Recordatorio**:  
La representación matemática de esta unidad es:  

$$ Z^{[l]} = W^{[l]}A^{[l-1]} + b^{[l]}$$

También puedes encontrar útil la función `np.dot()`. Si tus dimensiones no coinciden, imprimir `W.shape` puede ayudarte a depurar el error.

In [4]:
# FUNCTION: linear_forward

def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python dictionary containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    
    ### START CODE HERE ### (≈ 1 line of code)
    
    ### END CODE HERE ###
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [ ]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

**Salida esperada**:

<table style="width:35%">
  
  <tr>
    <td> **Z** </td>
    <td> [[ 3.26295337 -1.23429987]] </td> 
  </tr>
  
</table>

### 4.2 - Propagación Hacia Adelante con Activación  

En este cuaderno, utilizarás dos funciones de activación:  

- **Sigmoide**:  
  $$
  \sigma(Z) = \sigma(W A + b) = \frac{1}{1 + e^{-(W A + b)}}
  $$
  Te hemos proporcionado la función `sigmoid`. Esta función devuelve **dos** elementos:  
  - El valor de activación "`A`".  
  - Una "`cache`" que contiene "`Z`" (lo cual se usará en la función de propagación hacia atrás correspondiente).  

  Para utilizarla, simplemente llama:  
  ```python
  A, activation_cache = sigmoid(Z)
  ```

- **ReLU**:  
  La fórmula matemática de ReLU es:  
  $$
  A = RELU(Z) = \max(0, Z)
  $$
  Te hemos proporcionado la función `relu`. Esta función también devuelve **dos** elementos:  
  - El valor de activación "`A`".  
  - Una "`cache`" que contiene "`Z`" (lo cual se usará en la función de propagación hacia atrás correspondiente).  

  Para utilizarla, simplemente llama:  
  ```python
  A, activation_cache = relu(Z)
  ```

Para mayor comodidad, vas a agrupar dos funciones (**Lineal** y **Activación**) en una sola función (**LINEAR->ACTIVATION**). Por lo tanto, implementarás una función que realice el paso de propagación lineal seguido del paso de activación.  

### **Ejercicio**:  
Implementa la propagación hacia adelante de la capa *LINEAR->ACTIVATION*. La relación matemática es:  

$$A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} + b^{[l]})$$

donde la función de activación **"g"** puede ser `sigmoid()` o `relu()`.  

Usa `linear_forward()` y la función de activación correspondiente.

In [6]:
# FUNCTION: linear_activation_forward

def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python dictionary containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        
        ### END CODE HERE ###
    
    elif activation == "relu":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        
        ### END CODE HERE ###
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [ ]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

**Salida esperada**:
       
<table style="width:35%">
  <tr>
    <td> **With sigmoid: A ** </td>
    <td > [[ 0.96890023  0.11013289]]</td> 
  </tr>
  <tr>
    <td> **With ReLU: A ** </td>
    <td > [[ 3.43896131  0.        ]]</td> 
  </tr>
</table>


### **d) Modelo de $L$ Capas**  

Para hacer aún más conveniente la implementación de la Red Neuronal de $L$ capas, necesitarás una función que replique la función anterior (`linear_activation_forward` con **ReLU**) $L-1$ veces, y luego la siga con una llamada a `linear_activation_forward` con **Sigmoide**.  

### **Ejercicio**:  
Implementa la propagación hacia adelante del modelo mostrado arriba.  

### **Instrucciones**:  
En el código de abajo, la variable `AL` representará:  

$$A^{[L]} = \sigma(Z^{[L]}) = \sigma(W^{[L]} A^{[L-1]} + b^{[L]})$$

(Esta variable a veces también se denomina `Yhat`, es decir, $\hat{Y} $).  

### **Consejos**:  
- Usa las funciones que ya has implementado.  
- Utiliza un **bucle for** para replicar **[LINEAR->RELU] $ (L-1) $ veces**.  
- No olvides almacenar los valores intermedios en la lista `"caches"`. Para agregar un nuevo valor `c` a una lista, puedes usar:  
  ```python
  list.append(c)
  ```  


In [8]:
# FUNCTION: L_model_forward

def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_relu_forward() (there are L-1 of them, indexed from 0 to L-2)
                the cache of linear_sigmoid_forward() (there is one, indexed L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        ### START CODE HERE ### (≈ 2 lines of code)
        
        ### END CODE HERE ###
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    ### START CODE HERE ### (≈ 2 lines of code)
    
    ### END CODE HERE ###
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [ ]:
X, parameters = L_model_forward_test_case()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

<table style="width:40%">
  <tr>
    <td> **AL** </td>
    <td > [[ 0.17007265  0.2524272 ]]</td> 
  </tr>
  <tr>
    <td> **Length of caches list ** </td>
    <td > 2</td> 
  </tr>
</table>

¡Genial! Ahora tienes una propagación hacia adelante completa que toma la entrada $ X $ y produce un vector fila $A^{[L]} $ que contiene tus predicciones. Además, almacena todos los valores intermedios en `"caches"`.  

Usando $ A^{[L]} $, puedes calcular el costo de tus predicciones.

## **5 - Función de Costo**  

Ahora implementarás la propagación hacia adelante y hacia atrás. Necesitas calcular el costo para verificar si tu modelo realmente está aprendiendo.  

### **Ejercicio**:  
Calcula la función de costo **entropía cruzada** $J$ usando la siguiente fórmula:  

$$ J = -\frac{1}{m} \sum\limits_{i = 1}^{m} \left( y^{(i)}\log\left(a^{[L](i)}\right) + (1-y^{(i)})\log\left(1 - a^{[L](i)}\right) \right) \tag{7}$$

In [10]:
# FUNCTION: compute_cost

def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    ### START CODE HERE ### (≈ 1 lines of code)
    
    ### END CODE HERE ###
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    return cost

In [ ]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

**Salida esperada**:

<table>
    <tr>
    <td>**cost** </td>
    <td> 0.41493159961539694</td> 
    </tr>
</table>

## **6 - Módulo de Propagación Hacia Atrás**  

Al igual que en la propagación hacia adelante, implementarás funciones auxiliares para la propagación hacia atrás (**backpropagation**). Recuerda que la propagación hacia atrás se utiliza para calcular el **gradiente de la función de pérdida con respecto a los parámetros**.  

### **Recordatorio**  
<img src="images/backprop_kiank.png" style="width:650px;height:250px;">  
<caption><center> **Figura 3** : Propagación hacia adelante y hacia atrás para *LINEAR->RELU->LINEAR->SIGMOID* <br> *Los bloques morados representan la propagación hacia adelante, y los bloques rojos representan la propagación hacia atrás.*  </center></caption>  

---

### **Construcción de la Propagación Hacia Atrás**  
De manera similar a la propagación hacia adelante, construirás la propagación hacia atrás en **tres pasos**:  

1. **Propagación LINEAR hacia atrás**  
2. **Propagación LINEAR -> ACTIVATION hacia atrás**, donde ACTIVATION calculará la derivada de la función de activación **ReLU** o **sigmoide**.  
3. **[LINEAR -> RELU] $ \times (L-1) $ -> LINEAR -> SIGMOID hacia atrás** (modelo completo).  



### **6.1 - Propagación Lineal Hacia Atrás**  

Para la capa $ l $, la parte lineal se define como:  

$$
Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}
$$

(seguida por una activación).  

Supongamos que ya has calculado la derivada:  

$$
dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}
$$

Tu objetivo es obtener los gradientes $ (dW^{[l]}, db^{[l]}, dA^{[l-1]}) $.  

<img src="images/linearback_kiank.png" style="width:250px;height:300px;">  
<caption><center> **Figura 4** </center></caption>  

Los tres valores de salida $ (dW^{[l]}, db^{[l]}, dA^{[l-1]}) $ se calculan utilizando la entrada $ dZ^{[l]} $. Aquí están las fórmulas que necesitas:  

$$
dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}
$$

$$
db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)} \tag{9}
$$

$$
dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}
$$


**Exercise**: Use the 3 formulas above to implement linear_backward().

In [12]:
# FUNCTION: linear_backward

def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### START CODE HERE ### (≈ 3 lines of code)
    
    ### END CODE HERE ###
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [ ]:
# Set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

**Salida esperada**: 

<table style="width:90%">
  <tr>
    <td> **dA_prev** </td>
    <td > [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]] </td> 
  </tr> 
    <tr>
        <td> **dW** </td>
        <td > [[-0.10076895  1.40685096  1.64992505]] </td> 
    </tr> 
    <tr>
        <td> **db** </td>
        <td> [[ 0.50629448]] </td> 
    </tr> 
    
</table>



### **6.2 - Propagación Hacia Atrás con Activación Lineal**  

Ahora, crearás una función que combine dos funciones auxiliares:  
- **`linear_backward`** (propagación hacia atrás de la parte lineal).  
- **`linear_activation_backward`** (propagación hacia atrás de la activación).  

### **Funciones proporcionadas**  
Para ayudarte a implementar `linear_activation_backward`, se te han proporcionado dos funciones de propagación hacia atrás:  

- **`sigmoid_backward`**: Implementa la propagación hacia atrás para la unidad **Sigmoide**. Puedes llamarla de la siguiente manera:  

  ```python
  dZ = sigmoid_backward(dA, activation_cache)
  ```

- **`relu_backward`**: Implementa la propagación hacia atrás para la unidad **ReLU**. Puedes llamarla así:  

  ```python
  dZ = relu_backward(dA, activation_cache)
  ```

Si $g(.) $ es la función de activación, `sigmoid_backward` y `relu_backward` calculan:  

$$
dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}) \tag{11}
$$

### **Ejercicio**:  
Implementa la propagación hacia atrás para la capa **LINEAR->ACTIVATION** utilizando las funciones anteriores.

In [14]:
# FUNCTION: linear_activation_backward

def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        ### START CODE HERE ### (≈ 2 lines of code)
        
        ### END CODE HERE ###
        
    elif activation == "sigmoid":
        ### START CODE HERE ### (≈ 2 lines of code)
        
        ### END CODE HERE ###
    
    return dA_prev, dW, db

In [ ]:
AL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

**Salida esperada con sigmoid:**

<table style="width:100%">
  <tr>
    <td > dA_prev </td> 
           <td >[[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]] </td> 

  </tr> 
    <tr>
    <td > dW </td> 
           <td > [[ 0.10266786  0.09778551 -0.01968084]] </td> 
  </tr> 
    <tr>
    <td > db </td> 
           <td > [[-0.05729622]] </td> 
  </tr> 
</table>



**Salida esperada con relu**

<table style="width:100%">
  <tr>
    <td > dA_prev </td> 
           <td > [[ 0.44090989  0.        ]
 [ 0.37883606  0.        ]
 [-0.2298228   0.        ]] </td> 

  </tr> 
   <tr>
    <td > dW </td> 
           <td > [[ 0.44513824  0.37371418 -0.10478989]] </td> 
  </tr> 
    <tr>
    <td > db </td> 
           <td > [[-0.20837892]] </td> 
  </tr> 
</table>



### **6.3 - Propagación Hacia Atrás en un Modelo de $ L $ Capas**  

Ahora implementarás la función de propagación hacia atrás para toda la red neuronal. Recuerda que cuando implementaste la función `L_model_forward`, en cada iteración almacenaste un **cache** que contiene los valores de **(X, W, b, y Z)**.  

En el módulo de propagación hacia atrás, utilizarás estas variables almacenadas para calcular los gradientes. Por lo tanto, en la función `L_model_backward`, iterarás **hacia atrás a través de todas las capas ocultas**, comenzando desde la capa $ L $. En cada paso, usarás los valores almacenados para **retropropagar** a través de la capa correspondiente.  

La **Figura 5** muestra el proceso de propagación hacia atrás:  

<img src="images/mn_backward.png" style="width:450px;height:300px;">  
<caption><center>  **Figura 5** : Propagación hacia atrás  </center></caption>  

---

### **Inicialización de la Retropropagación**  

Para retropropagar a través de la red, sabemos que la salida es:  

$$
A^{[L]} = \sigma(Z^{[L]})
$$

Por lo tanto, tu código debe calcular la derivada del costo con respecto a $ A^{[L]} $, es decir:  

$$
dAL = \frac{\partial \mathcal{L}}{\partial A^{[L]}}
$$

Puedes calcularlo con la siguiente fórmula (derivada del costo con respecto a $ A^{[L]} $):  

```python
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) 
```

---

### **Iteración Hacia Atrás por las Capas**  

1. **Usa `dAL` en la función LINEAR->SIGMOID hacia atrás**  
   - Esto usará los valores almacenados en el cache de `L_model_forward`.  
2. **Utiliza un bucle `for` para iterar a través de todas las demás capas ocultas**  
   - En cada capa $ l $, utiliza la función **LINEAR->RELU hacia atrás**.  
   - Almacena cada `dA`, `dW`, y `db` en un **diccionario `grads`**.  

Para almacenar los gradientes en el diccionario `grads`, usa esta fórmula:  

$$
grads["dW" + str(l)] = dW^{[l]} \tag{15}
$$

Por ejemplo, para $ l = 3 $, esto almacenará $ dW^{[3]} $ en `grads["dW3"]`.  

---

### **Ejercicio**:  
Implementa la retropropagación para el modelo **[LINEAR->RELU] $ \times (L-1) $ -> LINEAR -> SIGMOID**

In [16]:
# FUNCTION: L_model_backward

def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    ### START CODE HERE ### (1 line of code)
    
    ### END CODE HERE ###
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "AL, Y, caches". Outputs: "grads["dAL"], grads["dWL"], grads["dbL"]
    ### START CODE HERE ### (approx. 2 lines)
    
    ### END CODE HERE ###
    
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 2)], caches". Outputs: "grads["dA" + str(l + 1)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        ### START CODE HERE ### (approx. 5 lines)
        
        ### END CODE HERE ###

    return grads

In [ ]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dA1 = "+ str(grads["dA1"]))

**Salida esperada**

<table style="width:60%">
  
  <tr>
    <td > dW1 </td> 
           <td > [[ 0.41010002  0.07807203  0.13798444  0.10502167]
 [ 0.          0.          0.          0.        ]
 [ 0.05283652  0.01005865  0.01777766  0.0135308 ]] </td> 
  </tr> 
    <tr>
    <td > db1 </td> 
           <td > [[-0.22007063]
 [ 0.        ]
 [-0.02835349]] </td> 
  </tr> 
  
  <tr>
  <td > dA1 </td> 
           <td > [[ 0.          0.52257901]
 [ 0.         -0.3269206 ]
 [ 0.         -0.32070404]
 [ 0.         -0.74079187]] </td> 

  </tr> 
</table>



### **6.4 - Actualización de Parámetros**  

En esta sección, actualizarás los parámetros del modelo utilizando el **descenso del gradiente**.

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{16}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{17}$$

donde $\alpha$ es la **tasa de aprendizaje**.  

Después de calcular los parámetros actualizados, guárdalos en el diccionario `parameters`.

### **Ejercicio**: Implementa `update_parameters()` para actualizar los parámetros utilizando el **descenso del gradiente**.  

### **Instrucciones**:  
Actualiza los parámetros aplicando el descenso del gradiente a cada $ W^{[l]} $ y $ b^{[l]} $ para $ l = 1, 2, ..., L $.

In [18]:
# FUNCTION: update_parameters

def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    ### START CODE HERE ### (≈ 3 lines of code)
    
    ### END CODE HERE ###
        
    return parameters

In [ ]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

**Salida esperada**:

<table style="width:100%"> 
    <tr>
    <td > W1 </td> 
           <td > [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]] </td> 
  </tr> 
    <tr>
    <td > b1 </td> 
           <td > [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]] </td> 
  </tr> 
  <tr>
    <td > W2 </td> 
           <td > [[-0.55569196  0.0354055   1.32964895]]</td> 
  </tr> 
    <tr>
    <td > b2 </td> 
           <td > [[-0.84610769]] </td> 
  </tr> 
</table>
